In [1]:
import numpy as np
import pandas as pd

# Lab 9 - Multi-layer Perceptron Forward Pass & Backpropagation

## Part I
For this exercise you will implement a simple 2-layer perceptron with the forward pass and the backpropagation to learn the weights

For the first part you'll build and train a 2-layer neural network that predicts the prices of houses, using the usual Boston housing dataset.

In [2]:
boston = pd.read_csv('data/BostonHousing.txt')
boston.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


As usual, consider the MEDV as your target variable. 
* Split the data into training, validation and testing (70,15,15)%
* Experiment with different number of neurons per layer for your network, using the validation set

In [3]:
# your code goes here
from sklearn.model_selection import train_test_split

X = boston.drop('medv', axis=1)
y = boston['medv']

features = X.shape[1]
predictions = 1

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.50, random_state=42)

In [4]:
class MyStandardScaler:
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
    
    def transform(self, X):
        return (X - self.mean_) / self.scale_
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    
X_train_arr = X_train.values
y_train_arr = y_train.values

scaler = MyStandardScaler()
X_train_normalized = np.array(scaler.fit_transform(X_train_arr))

In [5]:
# activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def identity(x):
    return x

def identity_prime(x):
    return 1

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return (y_pred - y_true)/y_true.size # can be used to mini-batch

In [6]:
class FCLayer():
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.input = None
        self.output = None
        self.output_without_activation = None
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.activation = activation
        self.activation_prime = activation_prime

    # forward propagation through the layer
    def forward_propagation(self, input_data):
        self.input = input_data.reshape(1, -1)
        self.output_without_activation = np.dot(self.input, self.weights) + self.bias
        self.output = self.activation(self.output_without_activation)
        return self.output

    # computes dE/dW (weights), dE/dB (bias) for a given delta_k, from the subsequent layer. Returns delta_j, from this layer.
    def backward_propagation(self, output_error, learning_rate):
        delta_k_activated = self.activation_prime(self.output_without_activation) * output_error
        delta_j = np.dot(delta_k_activated, self.weights.T)
        gradiente = np.dot(self.input.T, delta_k_activated)

        # update parameters
        self.weights -= learning_rate * gradiente
        self.bias -= learning_rate * delta_k_activated # 1 * delta_k_activated, because input is 1 for bias
        return delta_j

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.errors_during_train = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            self.errors_during_train.append(err)
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
# network
net = Network()
net.add(FCLayer(features, 15, sigmoid, sigmoid_prime))
net.add(FCLayer(15, predictions, identity, identity_prime))

# train
net.use(mse, mse_prime)
net.fit(X_train_normalized, y_train_arr, epochs=1000, learning_rate=0.08)

# test
out = net.predict(X_train_normalized)
print(out)

epoch 1/1000   error=45.882028
epoch 2/1000   error=26.028944
epoch 3/1000   error=15.396497
epoch 4/1000   error=16.403746
epoch 5/1000   error=15.742298
epoch 6/1000   error=13.398818
epoch 7/1000   error=9.773057
epoch 8/1000   error=9.959105
epoch 9/1000   error=8.152865
epoch 10/1000   error=7.938398
epoch 11/1000   error=7.268268
epoch 12/1000   error=7.098999
epoch 13/1000   error=6.175362
epoch 14/1000   error=7.462773
epoch 15/1000   error=7.508745
epoch 16/1000   error=9.194004
epoch 17/1000   error=6.673881
epoch 18/1000   error=6.550414
epoch 19/1000   error=5.891324
epoch 20/1000   error=5.678264
epoch 21/1000   error=6.134689
epoch 22/1000   error=5.539700
epoch 23/1000   error=5.179248
epoch 24/1000   error=5.397133
epoch 25/1000   error=4.774994
epoch 26/1000   error=4.778925
epoch 27/1000   error=5.038880
epoch 28/1000   error=5.063512
epoch 29/1000   error=4.823695
epoch 30/1000   error=4.802207
epoch 31/1000   error=3.437879
epoch 32/1000   error=5.103077
epoch 33/10